# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../final/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.44,80,97,0.89,US,1720569655
1,1,happy valley-goose bay,53.3168,-60.3315,18.17,100,100,1.54,CA,1720569955
2,2,tura,25.5198,90.2201,24.29,99,100,3.05,IN,1720569783
3,3,adamstown,-25.0660,-130.1015,20.09,58,99,4.65,PN,1720569956
4,4,daru,-9.0763,143.2092,25.63,82,80,7.16,PG,1720569957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,paveh,35.0434,46.3565,21.40,32,0,1.13,IR,1720569974
132,132,florence,43.7667,11.2500,24.61,70,0,2.06,IT,1720570058
173,173,trikala,39.5553,21.7675,26.15,32,0,1.57,GR,1720570091
180,180,chebba,35.2372,11.1150,25.89,66,0,1.68,TN,1720570098
208,208,tsagan aman,47.5639,46.7218,23.10,39,0,3.22,RU,1720570122
219,219,zaragoza,41.6561,-0.8773,26.38,64,0,3.09,ES,1720570130
221,221,wangqing,43.3218,129.7634,21.68,51,0,1.17,CN,1720570132
293,293,ironwood,46.4547,-90.1710,24.45,47,0,2.06,US,1720570190
332,332,astrakhan,46.3497,48.0408,22.93,38,0,4.00,RU,1720570221
358,358,ribeira brava,32.6500,-17.0667,22.35,85,0,0.64,PT,1720570241


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,paveh,IR,35.0434,46.3565,32,
132,florence,IT,43.7667,11.2500,70,
173,trikala,GR,39.5553,21.7675,32,
180,chebba,TN,35.2372,11.1150,66,
208,tsagan aman,RU,47.5639,46.7218,39,
219,zaragoza,ES,41.6561,-0.8773,64,
221,wangqing,CN,43.3218,129.7634,51,
293,ironwood,US,46.4547,-90.1710,47,
332,astrakhan,RU,46.3497,48.0408,38,
358,ribeira brava,PT,32.6500,-17.0667,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
paveh - nearest hotel: هتل ارم
florence - nearest hotel: Old bridge Apartment
trikala - nearest hotel: Αχίλλειον
chebba - nearest hotel: No hotel found
tsagan aman - nearest hotel: No hotel found
zaragoza - nearest hotel: Hotel Pilar Plaza
wangqing - nearest hotel: No hotel found
ironwood - nearest hotel: Comfort Inn
astrakhan - nearest hotel: Omega
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
timon - nearest hotel: Hotel Cocais
aripuana - nearest hotel: No hotel found
posto fiscal rolim de moura - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
26,paveh,IR,35.0434,46.3565,32,هتل ارم
132,florence,IT,43.7667,11.2500,70,Old bridge Apartment
173,trikala,GR,39.5553,21.7675,32,Αχίλλειον
180,chebba,TN,35.2372,11.1150,66,No hotel found
208,tsagan aman,RU,47.5639,46.7218,39,No hotel found
219,zaragoza,ES,41.6561,-0.8773,64,Hotel Pilar Plaza
221,wangqing,CN,43.3218,129.7634,51,No hotel found
293,ironwood,US,46.4547,-90.1710,47,Comfort Inn
332,astrakhan,RU,46.3497,48.0408,38,Omega
358,ribeira brava,PT,32.6500,-17.0667,85,Cheerfulway Bravamar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)